In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
import torch.nn as nn
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [4]:
df = pd.read_csv("cb_tweets.csv")
df = df.rename(columns={"tweet_text": "text", "cyberbullying_type": "label"})
df["label"] = df["label"].apply(lambda x: 0 if x == "not_cyberbullying" else 1)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42, stratify=df["label"]
)

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

class CyberbullyingDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.float)
        }

train_dataset = CyberbullyingDataset(train_texts, train_labels, tokenizer)
test_dataset = CyberbullyingDataset(test_texts, test_labels, tokenizer)


In [6]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)


In [7]:
class BERTClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        x = self.linear(x)
        return self.sigmoid(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier().to(device)


In [8]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()


In [9]:
epochs = 2

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device).unsqueeze(1)
        
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/2: 100%|█████████████████████████████████████████████████████████████████| 2385/2385 [2:21:30<00:00,  3.56s/it]


Epoch 1 Loss: 0.2662


Epoch 2/2: 100%|█████████████████████████████████████████████████████████████████| 2385/2385 [2:06:12<00:00,  3.18s/it]

Epoch 2 Loss: 0.2068


In [11]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        preds = (outputs > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=["non-bullying", "bullying"]))

acc = accuracy_score(y_true, y_pred)
print(f"✅ Test Accuracy: {acc:.4f}")

Evaluating: 100%|████████████████████████████████████████████████████████████████████| 597/597 [06:05<00:00,  1.63it/s]



📊 Classification Report:
              precision    recall  f1-score   support

non-bullying       0.76      0.50      0.61      1589
    bullying       0.91      0.97      0.94      7950

    accuracy                           0.89      9539
   macro avg       0.84      0.74      0.77      9539
weighted avg       0.88      0.89      0.88      9539

✅ Test Accuracy: 0.8915
